In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import re
#imputer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
#read in the data 
data = pd.read_csv('data.csv')
#remove the first column and the second column
data = data.drop(data.columns[0], axis=1)
data = data.drop(data.columns[0], axis=1)
#remove the "Processed-Title" column in data 
data = data.drop('Processed-Title', axis=1)

#### User-Age

In [3]:
#get any num" to num
data['User-Age'] = data['User-Age'].str.extract('(\d+)')
#get the data type to be int
data['User-Age'] = data['User-Age'].astype(float)

In [4]:
#get any age > 100 to NAN
data.loc[data['User-Age'] > 100, 'User-Age'] = np.nan
#get any age < 5 to NAN
data.loc[data['User-Age'] < 5, 'User-Age'] = np.nan

#create a pipeline to impute the missing values to median
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

#fit the pipeline to the User-Age column
pipeline.fit(data[['User-Age']])
#transform the User-Age column
data['User-Age'] = pipeline.transform(data[['User-Age']])

#summary statistics
data['User-Age'].unique()

array([ 33.,  21.,  31.,  14.,  29.,  11.,  27.,  22.,  34.,  25.,  70.,
        54.,  24.,  48.,  65.,  62.,  32.,  42.,  50.,  56.,  17.,  40.,
        35.,  52.,  26.,  63.,  60.,  28.,  55.,  30.,  51.,  46.,  59.,
        57.,  39.,  15.,  47.,  20.,  23.,  49.,  36.,  37.,  53.,  41.,
        75.,  69.,  61.,  16.,  43.,  45.,  64.,  18.,  44.,  68.,  71.,
        19.,  66.,  38.,  13.,  78.,  58.,  76.,  67.,  80.,  83.,  73.,
        72.,  74.,  12.,   9.,  10.,  77.,  97.,   8.,  90.,  79.,  81.,
        82., 100.,   7.,  93.,  99.,  84.])

#### User-Country

In [5]:
#get any str" to str
data['User-Country'] = data['User-Country'].str.extract('([a-zA-Z]+)')

data['User-Country'].unique().__len__()

101

In [6]:
country_corrections = {
    'usa': 'United States',
    'united': 'United Kingdom',  # Assuming 'united' might be shorthand for 'United Kingdom'
    'hong': 'Hong Kong',  # Assuming meant to be 'Hong Kong'
    'czech': 'Czech Republic',
    'costa': 'Costa Rica',
    'south': 'South Korea',  # Assuming it refers to 'South Korea', could also be 'South Africa'
    'ua': 'Ukraine',  # Common abbreviation
    'us': 'United States',
    'far': 'Faroe Islands',  # If it is meant to represent 'Faroe Islands'
    'rutherford': None,  # Not a country, likely invalid data
    'morgan': None,  # Not a country, likely a mistake
    'framingham': None,  # City in Massachusetts, not a country
    'worcester': None,  # City in Massachusetts, not a country
    'van': 'Vanuatu',  # Assuming it's an abbreviation for 'Vanuatu'
    'euskal': 'Spain',  # Refers to the Basque region, which is part of Spain
    'universe': None,  # Clearly invalid
    'quit': None,  # Likely invalid or a typo
    'phillipines': 'Philippines',  # Spelling correction
    'cayman': 'Cayman Islands',
    'caribbean': None,  # Not a single country, too broad
    'catalonia': 'Spain',  # Region in Spain, not a separate country
    'u': None  # Not clear what country this should refer to
    # Add more as necessary
}

# Replace incorrect country names using the mapping
data['User-Country'] = data['User-Country'].replace(country_corrections)

valid_countries = ['United States', 'Germany', 'Spain', 'Canada', 'Portugal', 'Australia', 'Indonesia', 'Japan', 'France',
                   'Taiwan', 'Italy', 'Malaysia', 'Sweden', 'New Zealand', 'Iran', 'Hong Kong', 'Romania', 'Netherlands',
                   'Austria', 'Singapore', 'Guatemala', 'India', 'Israel', 'Philippines', 'Greece', 'Czech Republic', 'Ireland',
                   'Switzerland', 'Finland', 'Brazil', 'Slovenia', 'Bulgaria', 'Costa Rica', 'Antarctica', 'Belgium', 'Uruguay',
                   'Egypt', 'Algeria', 'Mexico', 'Jamaica', 'South Korea', 'Denmark', 'China', 'Zimbabwe', 'Turkey', 'Belize',
                   'Cyprus', 'Ecuador', 'Burma', 'Qatar', 'Kuwait', 'Argentina', 'Iceland', 'Bermuda', 'Bahamas', 'Saudi Arabia',
                   'Nepal', 'Peru', 'Russia', 'Pakistan', 'Norway', 'Luxembourg', 'Dominican Republic', 'Albania', 'Macedonia',
                   'Kenya', 'Mauritius', 'Chile', 'Venezuela', 'Guinea', 'Poland', 'Honduras', 'Barbados', 'Ukraine', 'Yugoslavia',
                   'Thailand', 'Mozambique', 'Ghana', 'United Kingdom']  # Add all valid countries to this list

# Filter the DataFrame with the lowercased country names too, and replace lowercase country names with the correct ones
valid_countries_lower = [country.lower() for country in valid_countries]
data = data[data['User-Country'].isin(valid_countries_lower) |
            data['User-Country'].isin(valid_countries)]
data['User-Country'] = data['User-Country'].replace({country.lower(): country for country in valid_countries})


In [7]:
#data['User-Country'].unique()
data['User-Country'].unique().__len__()

76

In [8]:
#summarise every column
data.describe(include='all')

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher,User-City,User-State,User-Country,User-Age,Series-Title,Genre
count,25730.000000,25730,25730.000000,25730,25730,25730.000000,25730,25643,25730,25730,25730.000000,25730,25730
unique,NaN,758,NaN,565,268,NaN,229,4236,486,76,NaN,86,9
top,NaN,0971880107,NaN,Wild Animus,John Grisham,NaN,Warner Books,toronto,california,United States,NaN,ring,Fantasy
freq,NaN,556,NaN,556,2108,NaN,1557,296,3116,20554,NaN,2976,7092
mean,140323.671901,NaN,7.879829,NaN,NaN,1984.323397,NaN,NaN,NaN,NaN,34.699883,NaN,NaN
std,79990.125905,NaN,1.834800,NaN,NaN,164.753802,NaN,NaN,NaN,NaN,10.212067,NaN,NaN
min,51.000000,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,7.000000,NaN,NaN
25%,71705.250000,NaN,7.000000,NaN,NaN,1996.000000,NaN,NaN,NaN,NaN,29.000000,NaN,NaN
50%,138995.000000,NaN,8.000000,NaN,NaN,1999.000000,NaN,NaN,NaN,NaN,33.000000,NaN,NaN
75%,210587.000000,NaN,9.000000,NaN,NaN,2001.000000,NaN,NaN,NaN,NaN,38.000000,NaN,NaN


In [9]:
data.to_csv('cleaned_data.csv', index=False)